#### מגישות:
ירדן סמואל 

רננה פריד 

githubקישור ל:
https://github.com/renanafried/Matala1

In [106]:
import requests
from bs4 import BeautifulSoup

def get_cars(keywords):
    car_list = []
    base_url = "https://www.ad.co.il/car?keyword=" + keywords
    page_number = 1

    while page_number < 9:
        url = f"{base_url}&page={page_number}"
        response = requests.get(url)
        if response.status_code != 200:
            break  # Exit if the page doesn't load correctly
        
        try:
            results_page = BeautifulSoup(response.content, 'html.parser')
            cars = results_page.find_all('div', {'class': 'card-body p-md-3'})
            if not cars:
                break  # Exit if no more cars are found
            
            for car in cars:
                car_link = "https://www.ad.co.il" + car.find('a').get('href')
                car_name = car.find('a').get_text().strip()
                if keywords in car_name:
                    
                    car_list.append(car_link)
            
            page_number += 1  # Move to the next page
        except Exception as e:
            print(f"Error: {e}")
            break
    
    return car_list

In [107]:
get_cars("מאזדה")

['https://www.ad.co.il/ad/16189128',
 'https://www.ad.co.il/ad/16186152',
 'https://www.ad.co.il/ad/16189114',
 'https://www.ad.co.il/ad/16185806',
 'https://www.ad.co.il/ad/16186411',
 'https://www.ad.co.il/ad/16168929',
 'https://www.ad.co.il/ad/16167062',
 'https://www.ad.co.il/ad/16166014',
 'https://www.ad.co.il/ad/16162970',
 'https://www.ad.co.il/ad/16137890',
 'https://www.ad.co.il/ad/16128563',
 'https://www.ad.co.il/ad/15952860',
 'https://www.ad.co.il/ad/15910677',
 'https://www.ad.co.il/ad/16116498',
 'https://www.ad.co.il/ad/16102966',
 'https://www.ad.co.il/ad/16068875',
 'https://www.ad.co.il/ad/16086340',
 'https://www.ad.co.il/ad/16067160',
 'https://www.ad.co.il/ad/16065478',
 'https://www.ad.co.il/ad/16050602',
 'https://www.ad.co.il/ad/16045395',
 'https://www.ad.co.il/ad/15905182',
 'https://www.ad.co.il/ad/16041416',
 'https://www.ad.co.il/ad/16035021',
 'https://www.ad.co.il/ad/16022117',
 'https://www.ad.co.il/ad/16018006',
 'https://www.ad.co.il/ad/16084476',
 

In [108]:
import calendar
from datetime import datetime

def get_car_info(car_link):
    from datetime import datetime, date
    car_list = list()
    car_info={}
    response = requests.get(car_link)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        result_page = BeautifulSoup(response.content,'html.parser')
        table=result_page.find('table',{'class':'table table-sm mb-4'}).find('tbody')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    car_info[key] = value
                    if 'שנה' in car_info:
                        car_info['Year'] = car_info.pop('שנה')
                        car_info['Year'] = int(car_info['Year'])
                    if 'יד' in car_info:
                        car_info['Hand'] = car_info.pop('יד')
                        car_info['Hand'] = int(car_info['Hand'])
                    if 'ת. הילוכים' in car_info:
                        car_info['Gear'] = car_info.pop('ת. הילוכים') 
                    if 'נפח' in car_info:
                        car_info['capacity_Engine'] = car_info.pop('נפח').replace(',', '')
                        car_info['capacity_Engine'] = int(car_info['capacity_Engine'])
                    if 'סוג מנוע' in car_info:
                        car_info['Engine_type'] = car_info.pop('סוג מנוע')
                    if 'בעלות קודמת' in car_info:
                        car_info['Prev_ownership'] = car_info.pop('בעלות קודמת')
                    if 'בעלות נוכחית' in car_info:
                        car_info['Curr_ownership'] = car_info.pop('בעלות נוכחית')
                    if 'אזור' in car_info:
                        car_info['Area'] = car_info.pop('אזור')
                    if 'עיר' in car_info:
                        car_info['City'] = car_info.pop('עיר')
                    if 'צבע' in car_info:
                        car_info['Color'] = car_info.pop('צבע')
                    if 'ק"מ' in car_info:
                        car_info['Km'] = car_info.pop('ק"מ').replace(',', '')
                        car_info['Km'] = int(car_info['Km']) 
                    if 'טסט עד' in car_info:
                        test_date_str = car_info.pop('טסט עד')
                        test_date = datetime.strptime(test_date_str, '%m/%Y')
                        # Get the last day of the month
                        last_day_of_month = calendar.monthrange(test_date.year, test_date.month)[1]
                        test_date = test_date.replace(day=last_day_of_month)
                        today = datetime.now()
                        days_until_test = (test_date - today).days
                        car_info['Test'] = int(days_until_test)
                        
                    
        car_info["manufactor"] = "מאזדה"
    
        model_and_price= result_page.find_all('h2',{'class':'card-title'})
        car_info['model']=model_and_price[0].get_text()
        car_info['Price']=float(model_and_price[1].get_text().strip('₪').replace(',', ''))
        
        picture= result_page.find_all('div',{'class':'justify-content-center px-1'})
        car_info['Pic_num']=int(len(picture))       
        
        Repub_date_and_Cre_date= result_page.find_all('div',{'class':'px-3'})
        cre_date=Repub_date_and_Cre_date[0].get_text().split()
        Repub_date=Repub_date_and_Cre_date[1].get_text().split()
        car_info['Cre_date']= cre_date[2].replace('/', '-')
        car_info['Repub_date']=Repub_date[3].replace('/', '-')
        car_info['Cre_date'] = datetime.strptime(car_info['Cre_date'], '%d-%m-%Y')
        car_info['Repub_date'] = datetime.strptime(car_info['Repub_date'], '%d-%m-%Y')

        car_info['description']=result_page.find('p',{'class':'text-word-break'}).get_text()
            
        return car_info
    except:
        return car_info
    

In [109]:
get_car_info('https://www.ad.co.il/ad/16186152')

{'Year': 2008,
 'Hand': 3,
 'Gear': 'אוטומטית',
 'capacity_Engine': 2000,
 'Engine_type': 'בנזין',
 'Km': 195000,
 'Test': 146,
 'Color': 'שחור',
 'Prev_ownership': 'פרטית',
 'Curr_ownership': 'פרטית',
 'Area': 'בית שמש והסביבה',
 'City': 'בית שמש',
 'manufactor': 'מאזדה',
 'model': 'מאזדה 6',
 'Price': 16000.0,
 'Pic_num': 4,
 'Cre_date': datetime.datetime(2024, 5, 16, 0, 0),
 'Repub_date': datetime.datetime(2024, 6, 4, 0, 0),
 'description': 'מאזדה 6 שנת 2008 במצב מצוין מכאנית\nטסט עד סוף חודש 9 \n195,000 אלף קילומטר גמיש במחיר'}

In [110]:
def get_all_cars(keywords):
    results = list()
    all_cars = get_cars(keywords)
    for car in all_cars:
        results.append(get_car_info(car))
    return(results)

In [111]:
mazda_cars = get_all_cars("מאזדה")
mazda_cars

[{'Year': 2017,
  'Hand': 2,
  'Gear': 'אוטומטית',
  'capacity_Engine': 2000,
  'Engine_type': 'בנזין',
  'Km': 115000,
  'Test': 388,
  'Color': 'כחול',
  'Prev_ownership': 'ליסינג',
  'Curr_ownership': 'פרטית',
  'Area': 'רמת גן - גבעתיים',
  'City': 'גבעתיים',
  'manufactor': 'מאזדה',
  'model': 'מאזדה 5',
  'Price': 70000.0,
  'Pic_num': 4,
  'Cre_date': datetime.datetime(2024, 6, 5, 0, 0),
  'Repub_date': datetime.datetime(2024, 6, 5, 0, 0),
  'description': 'מאזדה CX5 שנת 2017 **צמיגים דאנלופ חדשים 3,200 ₪** ליסינג פרטי נהג יחיד. מטופל במוסך קבוע. הסטוריית טיפולים. חלונות חשמליים. חיישן רוורס. מצלמת רוורס. מערכת מולטימדיה. עם או בלי טסט לשנה'},
 {'Year': 2008,
  'Hand': 3,
  'Gear': 'אוטומטית',
  'capacity_Engine': 2000,
  'Engine_type': 'בנזין',
  'Km': 195000,
  'Test': 146,
  'Color': 'שחור',
  'Prev_ownership': 'פרטית',
  'Curr_ownership': 'פרטית',
  'Area': 'בית שמש והסביבה',
  'City': 'בית שמש',
  'manufactor': 'מאזדה',
  'model': 'מאזדה 6',
  'Price': 16000.0,
  'Pic_num':

### intיצירת דאטה פריים של הנתונים והשמה של משתנים קטגוריאליים ו 

In [112]:
import pandas as pd
import numpy as np

data= pd.DataFrame(mazda_cars)
data['Gear'] = pd.Categorical(data['Gear'])
data['Engine_type'] = pd.Categorical(data['Engine_type'])
data['Prev_ownership'] = pd.Categorical(data['Prev_ownership'])
data['Curr_ownership'] = pd.Categorical(data['Curr_ownership'])
data['Test'] = data['Test'].astype('Int64')
data['Km'] = data['Km'].astype('Int64')

data

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,manufactor,model,Price,Pic_num,Cre_date,Repub_date,description
0,2017,2,אוטומטית,2000,בנזין,115000,388,כחול,ליסינג,פרטית,רמת גן - גבעתיים,גבעתיים,מאזדה,מאזדה 5,70000.0,4,2024-06-05,2024-06-05,"מאזדה CX5 שנת 2017 **צמיגים דאנלופ חדשים 3,200..."
1,2008,3,אוטומטית,2000,בנזין,195000,146,שחור,פרטית,פרטית,בית שמש והסביבה,בית שמש,מאזדה,מאזדה 6,16000.0,4,2024-05-16,2024-06-04,מאזדה 6 שנת 2008 במצב מצוין מכאנית\nטסט עד סוף...
2,2007,4,אוטומטית,1600,בנזין,189000,146,אפור,פרטית,פרטית,באר שבע והסביבה,אופקים,מאזדה,מאזדה 3,8000.0,3,2024-06-05,2024-06-05,‏‪‪הרכב במצב טוב עבר טיפול לפני חודשיים לקחת ו...
3,2015,4,אוטומטית,1500,בנזין,152000,<NA>,שחור,פרטית,פרטית,עכו - נהריה,נהריה,מאזדה,מאזדה 3,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ..."
4,2022,2,אוטומטית,2000,בנזין,30000,327,שחור,פרטית,פרטית,נתניה והסביבה,כפר יונה,מאזדה,מאזדה 3,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,2008,5,אוטומטית,1600,בנזין,237000,<NA>,כחול כהה,פרטית,פרטית,מודיעין והסביבה,שוהם,מאזדה,מאזדה 3,11000.0,3,2023-04-24,2023-04-24,"מאזדה 3 השנתון הכי אמין שיש,הרכב במצב מעולהביח..."
380,2008,4,אוטומטית,2000,בנזין,243000,<NA>,כסוף,פרטית,פרטית,ראש העין והסביבה,אלעד,מאזדה,מאזדה 5,22000.0,2,2023-04-19,2023-04-19,"הרכב נוסע כמו חדש, מנוע חדש באחריות הוחלף לפני..."
381,2009,2,אוטומטית,1600,בנזין,245000,<NA>,ברונזה מטאלי,ליסינג,פרטית,ראש העין והסביבה,אלעד,מאזדה,מאזדה 3,14800.0,3,2023-04-19,2023-04-19,הרכב אצלי 10 שנים מטופל במוסך מגדל פתח תקווה ע...
382,2006,2,אוטומטית,1600,בנזין,174000,<NA>,אפור,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,מאזדה,מאזדה 3,13800.0,2,2023-03-28,2023-03-28,רכב כמו חדש. עם מגיני רוח חיישן רורס


### סינון השנים הרלוונטיות 2010-2024

In [113]:
data = data.loc[(data['Year'] >= 2010) & (data['Year'] <= 2024)]
data

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,manufactor,model,Price,Pic_num,Cre_date,Repub_date,description
0,2017,2,אוטומטית,2000,בנזין,115000,388,כחול,ליסינג,פרטית,רמת גן - גבעתיים,גבעתיים,מאזדה,מאזדה 5,70000.0,4,2024-06-05,2024-06-05,"מאזדה CX5 שנת 2017 **צמיגים דאנלופ חדשים 3,200..."
3,2015,4,אוטומטית,1500,בנזין,152000,<NA>,שחור,פרטית,פרטית,עכו - נהריה,נהריה,מאזדה,מאזדה 3,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ..."
4,2022,2,אוטומטית,2000,בנזין,30000,327,שחור,פרטית,פרטית,נתניה והסביבה,כפר יונה,מאזדה,מאזדה 3,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...
5,2012,3,אוטומטית,2000,בנזין,209,85,שחור,פרטית,פרטית,קריות,קרית אתא,מאזדה,מאזדה 3,29900.0,5,2024-04-06,2024-04-07,הרכב במצב חדש שמור ומטופל לפי הספר
7,2018,3,אוטומטית,2000,בנזין,52000,297,לבן פנינה,אחר,פרטית,אשדוד - אשקלון,אשקלון,מאזדה,מאזדה 3,74500.0,7,2024-04-03,2024-04-03,מאזדה 3 האצ׳בק ספורט 2018 למכירה\nנקנתה לפני ש...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2023,1,אוטומטית,2000,בנזין,150,<NA>,לבן פנינה,פרטית,פרטית,רמת השרון - הרצליה,הרצליה,מאזדה,מאזדה 3,155000.0,2,2023-06-08,2023-06-08,cx30 רכב חדש מניילון עם 150 קילומטר. נמכר עקב ...
374,2015,3,אוטומטית,1500,בנזין,125000,<NA>,אפור,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,מאזדה,מאזדה 2,46000.0,3,2023-01-03,2023-05-23,רכב שמור וחסכוני מטופל בזמן ובקפידה לאחר ברקסי...
375,2018,2,אוטומטית,2000,בנזין,104000,<NA>,לבן,פרטית,פרטית,פרדס חנה - כרכור,חריש,מאזדה,מאזדה 3,85000.0,3,2023-05-23,2023-05-23,"הדגם המובחר והמוצלח עם גג נפתח, ג'אנטים 18, מו..."
377,2014,3,טיפטרוניק,1500,בנזין,144500,<NA>,כחול כהה מטאלי,ליסינג,פרטית,רעננה - כפר סבא,כפר סבא,מאזדה,מאזדה 3,45000.0,7,2023-05-14,2023-05-14,מזדה 3 2014 מצב מצוין\n טסט עד חודש 8 \n144500...


### בדיקת המשתנים לטיפוס הרצוי

In [114]:
data.dtypes

Year                        int64
Hand                        int64
Gear                     category
capacity_Engine             int64
Engine_type              category
Km                          Int64
Test                        Int64
Color                      object
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
manufactor                 object
model                      object
Price                     float64
Pic_num                     int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
description                object
dtype: object

In [115]:
#data.to_csv('C:\\Users\\רננה\\Desktop\\כריית וניתוח נתונים\\matala1_data.csv', index=False, encoding='utf-8-sig')

### רשות - יצירת מדד היצע מנתונים של משרד התחבורה


In [116]:
base_url = 'https://data.gov.il/api/3/action/datastore_search'
resource_id = '5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6'
limit = 1000  # Number of records to fetch per request
offset = 0  # Starting offset for pagination

all_records = []

while True:
    url = f"{base_url}?resource_id={resource_id}&limit={limit}&offset={offset}"
    response = requests.get(url)
    if response.status_code == 200:
        data1 = response.json()
        records = data1.get('result', {}).get('records', [])
        
        if not records:
            break # If no more records are returned, break the loop

        all_records.extend(records)
        offset += limit
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")
        break

df = pd.DataFrame(all_records)

print(f"Total records fetched: {len(df)}")


df_filtered = df[(df['tozar'].str.strip() == 'מזדה') & (df['shnat_yitzur'].between(2010, 2024))].copy()
columns = ['tozar', 'shnat_yitzur', 'mispar_rechavim_pailim', 'kinuy_mishari']
df_filtered = df_filtered[columns]

df_filtered


Total records fetched: 85870


,tozar,shnat_yitzur,mispar_rechavim_pailim,kinuy_mishari
38,מזדה,2021,10,CX-30
96,מזדה,2015,659,MAZDA 2
170,מזדה,2017,115,MAZDA 2
216,מזדה,2017,9,MAZDA 2
240,מזדה,2013,2,MAZDA3
...,...,...,...,...
85729,מזדה,2018,1,CX-5
85750,מזדה,2011,8,MAZDA 5
85757,מזדה,2019,2,CX-30
85824,מזדה,2022,5,MAZDA MX-5


In [117]:
df_filtered = df_filtered.dropna(subset=['shnat_yitzur', 'kinuy_mishari', 'mispar_rechavim_pailim'])
df_filtered['mispar_rechavim_pailim'] = pd.to_numeric(df_filtered['mispar_rechavim_pailim'], errors='coerce')
    
grouped = df_filtered.groupby(['shnat_yitzur', 'kinuy_mishari'])['mispar_rechavim_pailim'].sum().reset_index(name='Supply_score')
grouped

,shnat_yitzur,kinuy_mishari,Supply_score
0,2010,BT 50,0
1,2010,BT-50,47
2,2010,MAZDA 2,4974
3,2010,MAZDA 3,16508
4,2010,MAZDA 3 SPORT,729
...,...,...,...
153,2024,MAZDA3,1080
154,2024,MAZDA6,232
155,2024,MX 30,3
156,2024,MX 5,2


In [118]:
grouped['kinuy_mishari'].unique()

array(['BT 50', 'BT-50', 'MAZDA 2', 'MAZDA 3', 'MAZDA 3 SPORT', 'MAZDA 5',
       'MAZDA 6', 'MX 5', 'MX-5', 'MX-5 RHT', 'MX-5RHT', 'CX-5', 'CX5',
       'MAZDA 6 WAGON', 'MAZDA3', 'MAZDA 6 SW', 'CX-3', 'CX-30', 'MAZDA6',
       'CX 9', 'MAZDA CX-3', 'MAZDA CX-30', 'MAZDA CX-5', 'MAZDA MX-5',
       'MX-30', 'CX-50', 'MAZDA  3', 'MAZDA 2 HYBRID', 'MAZDA MX-30',
       'MX30', 'CX-60', 'MAZDA CX-90', 'MX 30', 'MAZDA2'], dtype=object)

In [119]:
data['model'].unique()

array(['מאזדה 5', 'מאזדה 3', 'מאזדה 6', 'מאזדה 2'], dtype=object)

In [120]:
import re

# Regular expression pattern to extract model names
pattern = r'\b(' + '|'.join(map(re.escape, [
    'BT 50', 'BT-50', 'MAZDA 2', 'MAZDA 3', 'MAZDA 3 SPORT', 'MAZDA 5',
    'MAZDA 6', 'MX 5', 'MX-5', 'MX-5 RHT', 'MX-5RHT', 'CX-5', 'CX5',
    'MAZDA 6 WAGON', 'MAZDA3', 'MAZDA 6 SW', 'CX-3', 'CX-30', 'MAZDA6',
    'CX 9', 'MAZDA CX-3', 'MAZDA CX-30', 'MAZDA CX-5', 'MAZDA MX-5',
    'MX-30', 'CX-50', 'MAZDA  3', 'MAZDA 2 HYBRID', 'MAZDA MX-30',
    'MX30', 'CX-60', 'MAZDA CX-90', 'MX 30', 'MAZDA2'
])) + r')\b'

# Extract model name from description column
data = data.copy()  # Make a copy to avoid the warning
data.loc[:, 'Model_Name'] = data['description'].str.extract(pattern, flags=re.IGNORECASE)

# Use the value from the "model" column if there is no description or the extraction fails
data.loc[:, 'Model_Name'] = data['Model_Name'].fillna(data['model'])

model_mapping = {
    'BT 50': 'BT-50',
    'MAZDA 3 SPORT': 'MAZDA 3',
    'MX 5': 'MX-5',
    'MX-5 RHT': 'MX-5',
    'MX-5RHT': 'MX-5',
    'MAZDA 6 WAGON': 'MAZDA 6',
    'MAZDA3': 'MAZDA 3',
    'MAZDA 6 SW': 'MAZDA 6',
    'CX-3': 'CX-3',
    'CX-30': 'CX-30',
    'MAZDA6': 'MAZDA 6',
    'CX 9': 'CX-9',
    'MAZDA CX-3': 'CX-3',
    'MAZDA CX-30': 'CX-30',
    'MAZDA CX-5': 'CX-5',
    'MAZDA MX-5': 'MX-5',
    'MX-30': 'MX-30',
    'CX-50': 'CX-50',
    'MAZDA  3': 'MAZDA 3',
    'MAZDA 2 HYBRID': 'MAZDA 2',
    'MAZDA MX-30': 'MX-30',
    'MX30': 'MX-30',
    'CX-60': 'CX-60',
    'MAZDA CX-90': 'CX-90',
    'MX 30': 'MX-30',
    'MAZDA2': 'MAZDA 2',
    'מאזדה 2': 'MAZDA 2',
    'מאזדה 6': 'MAZDA 6',
    'מאזדה 3': 'MAZDA 3',
    'מאזדה 5': 'MAZDA 5',
}

# Apply the mapping to create a new column with general model names
data.loc[:, 'General_Model_Name'] = data['Model_Name'].map(model_mapping)

# Perform the merge
merged_data = pd.merge(data, grouped, 
                       left_on=['General_Model_Name', 'Year'], 
                       right_on=['kinuy_mishari', 'shnat_yitzur'], 
                       how='left')

merged_data['Supply_score'] = merged_data['Supply_score'].astype('Int64')

merged_data.head(3)


,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,...,Price,Pic_num,Cre_date,Repub_date,description,Model_Name,General_Model_Name,shnat_yitzur,kinuy_mishari,Supply_score
0,2017,2,אוטומטית,2000,בנזין,115000,388,כחול,ליסינג,פרטית,...,70000.0,4,2024-06-05,2024-06-05,"מאזדה CX5 שנת 2017 **צמיגים דאנלופ חדשים 3,200...",מאזדה 5,MAZDA 5,NaN,NaN,<NA>
1,2015,4,אוטומטית,1500,בנזין,152000,<NA>,שחור,פרטית,פרטית,...,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ...",מאזדה 3,MAZDA 3,2015.0,MAZDA 3,6832
2,2022,2,אוטומטית,2000,בנזין,30000,327,שחור,פרטית,פרטית,...,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...,מאזדה 3,MAZDA 3,2022.0,MAZDA 3,330


In [121]:
columns_to_drop = ['Model_Name', 'General_Model_Name','shnat_yitzur','kinuy_mishari']
merged_data.drop(columns=columns_to_drop, inplace=True)
merged_data

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,manufactor,model,Price,Pic_num,Cre_date,Repub_date,description,Supply_score
0,2017,2,אוטומטית,2000,בנזין,115000,388,כחול,ליסינג,פרטית,רמת גן - גבעתיים,גבעתיים,מאזדה,מאזדה 5,70000.0,4,2024-06-05,2024-06-05,"מאזדה CX5 שנת 2017 **צמיגים דאנלופ חדשים 3,200...",<NA>
1,2015,4,אוטומטית,1500,בנזין,152000,<NA>,שחור,פרטית,פרטית,עכו - נהריה,נהריה,מאזדה,מאזדה 3,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ...",6832
2,2022,2,אוטומטית,2000,בנזין,30000,327,שחור,פרטית,פרטית,נתניה והסביבה,כפר יונה,מאזדה,מאזדה 3,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...,330
3,2012,3,אוטומטית,2000,בנזין,209,85,שחור,פרטית,פרטית,קריות,קרית אתא,מאזדה,מאזדה 3,29900.0,5,2024-04-06,2024-04-07,הרכב במצב חדש שמור ומטופל לפי הספר,4376
4,2018,3,אוטומטית,2000,בנזין,52000,297,לבן פנינה,אחר,פרטית,אשדוד - אשקלון,אשקלון,מאזדה,מאזדה 3,74500.0,7,2024-04-03,2024-04-03,מאזדה 3 האצ׳בק ספורט 2018 למכירה\nנקנתה לפני ש...,4543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2023,1,אוטומטית,2000,בנזין,150,<NA>,לבן פנינה,פרטית,פרטית,רמת השרון - הרצליה,הרצליה,מאזדה,מאזדה 3,155000.0,2,2023-06-08,2023-06-08,cx30 רכב חדש מניילון עם 150 קילומטר. נמכר עקב ...,163
228,2015,3,אוטומטית,1500,בנזין,125000,<NA>,אפור,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,מאזדה,מאזדה 2,46000.0,3,2023-01-03,2023-05-23,רכב שמור וחסכוני מטופל בזמן ובקפידה לאחר ברקסי...,3349
229,2018,2,אוטומטית,2000,בנזין,104000,<NA>,לבן,פרטית,פרטית,פרדס חנה - כרכור,חריש,מאזדה,מאזדה 3,85000.0,3,2023-05-23,2023-05-23,"הדגם המובחר והמוצלח עם גג נפתח, ג'אנטים 18, מו...",4543
230,2014,3,טיפטרוניק,1500,בנזין,144500,<NA>,כחול כהה מטאלי,ליסינג,פרטית,רעננה - כפר סבא,כפר סבא,מאזדה,מאזדה 3,45000.0,7,2023-05-14,2023-05-14,מזדה 3 2014 מצב מצוין\n טסט עד חודש 8 \n144500...,6917


### יצירת CSV

In [122]:
merged_data.to_csv('C:\\Users\\רננה\\Desktop\\כריית וניתוח נתונים\\matala1_merged_data.csv', index=False, encoding='utf-8-sig')